In [80]:
import math
import numpy as np
import copy
from scipy import integrate
N = 3
a = 0.6 - 3/N
b = 2 - N/13
dl = (b - a) / 5
mu1 = 15 / (N+3)
mu2 = -6 * N / 21
VarInt = 4
n = 4
eps = 0.000001

def kx(x):
    return (4 - 0.1*x) / (x**2 + N/16)

def qx(x):
    return (x+5) / (x**2 + 0.9 * N)

def fx(x):
    return (N+x) / 3.5

def phi0(x):
    return mu1 + (mu2-mu1) * math.sin(math.pi * (x - a) / (2 * (b - a)))

def derPhi0(x):
    return (mu2-mu1) * math.pi * 2 *(b - a) * math.cos((math.pi * (x - a) / (2 * (b - a))))
    
def phiK(x, k):
    return math.sin(math.pi * k * (x - a) /  (b - a))
                    
def derPhiK(x, k):
    return k * math.pi * (b - a) * math.cos(k * math.pi * (x - a) / (b - a))

@functools.cache
def thetaA(x, i, j):
    return kx(x) * derPhiK(x, i) * derPhiK(x,j) + qx(x) * phiK(x, i) * phiK(x, j) 
#посчитать с помощью интеграла встроенного
@functools.cache
def thetaB(x, i, j = 0):
    return fx(x) * phiK(x, i) - qx(x) * phi0(x) * phiK(x, i) - kx(x) * derPhiK(x, i) * derPhi0(x)

In [81]:
def GaussIntegral(i, j, f, a0, b0):
    #поделить на части и для каждой пересчитать
    c = [0.34785484, 0.65214516, 0.65214516,0.34785484]
    t = [-0.86113631, -0.33998104, 0.33998104, 0.86113631]
    points = [(b0+a0)/2 + (b0-a0)/2 * t[i] for i in range(VarInt)]
    return (b0 - a0) / 2 * (c[0] * f(points[0], i, j) + c[1] * f(points[1], i, j) + c[2] * f(points[2], i, j) + c[3] * f(points[3], i, j))

In [82]:
#смена строк
def swap_row(a, i, j):
  temp = copy.copy(A[i])
  A[i] = A[j]
  A[j] = temp

#прямой ход решения
def decision_sle_direct_move(A):
  for k in range(0, 4, 1):
    # вернём индекс максимального элемента по модулю из столбца
    index_max = np.argmax(abs(A[k:, k])) + k
    #меняем строки местами
    swap_row(A, k, index_max)

    A[k] /= A[k][k]
    for i in range(k + 1, n, 1):
      A[i] = A[i] - A[k] * A[i][k]
  return A


#обратный ход, нахождение X
def decision_sle_reverse_move(A, X, F):
  X[n - 1] = F[n - 1]
  for k in reversed(range(0, n - 1, 1)):
    sum_row = 0
    for i in reversed(range(0, n, 1)):
      if (A[k][i] == 0.0):
        continue
      if (X[i] is not None):
        sum_row += A[k][i] * X[i]
    X[k] = F[k] - sum_row
  return X

def print_array(A):
  for i in range(0, len(A), 1):
    for j in range(0, len(A[i]), 1):
      print("%.4f     " % A[i][j], end="")
    print("")
    
def print_vector(A):
    for j in range(0, len(A), 1):
      print("%.4f     " % A[j], end="")
    print("")

In [85]:
A = [[0] * VarInt for _ in range(VarInt)]
ATrue = [[0] * VarInt for _ in range(VarInt)]
B = [0] * VarInt
for i in range(VarInt):
    for j in range(VarInt):
        a0, b0 = a,b
        resultPrev = GaussIntegral(i+1,j+1,thetaA, a0, b0)
        resultLast = GaussIntegral(i+1,j+1,thetaA, a0, (b0 - a0)/ 2) +  GaussIntegral(i+1,j+1,thetaA, (b0 - a0)/ 2, b0)
        delta = (b0 - a0) / 3
        count = 3
        while abs(resultLast - resultPrev) > eps:
            resultPrev = resultLast
            resultLast = 0
            for l in range(count):
                resultLast += GaussIntegral(i+1,j+1,thetaA, a0 + l * delta, a0 + (l+1)*delta)
            count += 1
            delta = (b0-a0)/ count

        A[i][j] = resultLast
        # ATrue[i][j] = integrate.quad(thetaA, a,b, args=(i,j))
        ATrue[i][j] = integrate.quad(thetaA, a, b, args=(i + 1, j + 1))[0]
    
    a0, b0 = a,b
    resultPrev = GaussIntegral(i+1,0,thetaB, a0, b0)
    resultLast = GaussIntegral(i+1,0,thetaB, a0, (b0 - a0)/ 2) +  GaussIntegral(i+1,0,thetaB, (b0 - a0)/ 2, b0)
    delta = (b0 - a0) / 3
    count = 3
    while abs(resultLast - resultPrev) > eps:
        resultPrev = resultLast
        resultLast = 0
        for l in range(count):
            resultLast += GaussIntegral(i+1,0,thetaB, a0 + l * delta, a0 + (l+1)*delta)
        count += 1
        delta = (b0-a0)/ count
    B[i] = resultLast
    
# print_array(A)
# print_array(B)
A = copy.copy(A)
B = np.array(B)
C = np.array([None, None, None, None])


print_array(A)
print("----")
print_array(ATrue)
print("----")

print_vector(B)

#решаем СЛАУ
A = np.column_stack((A, B))
A = decision_sle_direct_move(A)
# print("Полученная трапецевидная (расширенная) матрица A:")
# print_array(A)

B = A[:, 4]
A = A[:, :4]
# обратное ход
C = decision_sle_reverse_move(A, C, B)
print("\nПолученное решение:", end="")
print("\n\nX = ", C, "\n\n")





#выводим значения в точках
durX = a
for i in range(6):
    unx = phi0(durX) 
    for j in range(VarInt):
        unx += C[j] * phiK(durX, j+1)
    print(f"u{i+1}(x) = {unx}")
    durX += dl

480.9371     393.3355     -50.5716     -307.9333     
393.3355     1560.7935     1179.9113     106.4829     
-50.5716     1179.9113     3780.8949     2692.9702     
-307.9333     106.4829     2692.9702     7005.1449     
----
[[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)], [(0.0, 0.0), (480.9370520028138, 2.3199172183990813e-11), (393.3354658693835, 5.00366222414006e-10), (-50.571570753096296, 2.0219517934960685e-09)], [(0.0, 0.0), (393.3354658693835, 5.003636819114754e-10), (1560.7934908617565, 2.4506991713746373e-09), (1179.9113060275563, 5.5816090638316496e-09)], [(0.0, 0.0), (-50.57157075309635, 2.021949888769692e-09), (1179.9113060275563, 5.5816090638316496e-09), (3780.8949108194747, 2.7459285166855918e-09)]]
----
3265.3642     1381.9933     -1240.9889     -2672.1075     

Полученное решение:

X =  [7.765114298294392 -1.2523677268322204 0.24989956075855818
 -0.11714053062490005] 


u1(x) = 2.5
u2(x) = 5.004548113431411
u3(x) = 7.140180200227771
u4(x) = 7.646905668134211
u5(x) =